<a href="https://colab.research.google.com/github/lenahdtr/fallstudie_lkws/blob/main/Stufe3l1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie- LKW Zuteilung - Stufe 3.1


In [ ]:
! pip install -q pyscipopt

In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

# Daten laden

*Daten aus den Tabellen, die hard codiert werden:*

In [ ]:
routes_data = [
    {"route_id": "t-4", "distance_total": 250.0, "distance_toll": 150.0, "starttime": 0.28125, "endtime": 0.71875},
    {"route_id": "t-5", "distance_total": 250.0, "distance_toll": 150.0, "starttime": 0.2708333333333333, "endtime": 0.7083333333333334},
    {"route_id": "t-6", "distance_total": 250.0, "distance_toll": 150.0, "starttime": 0.25, "endtime": 0.6875},

    {"route_id": "s-1", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.22916666666666666, "endtime": 0.6458333333333334},
    {"route_id": "s-2", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.25, "endtime": 0.6666666666666666},
    {"route_id": "s-3", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.375, "endtime": 0.6666666666666666},
    {"route_id": "s-4", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.2708333333333333, "endtime": 0.6875},

    {"route_id": "w1", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.22916666666666666, "endtime": 0.6458333333333334},
    {"route_id": "w2", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.3333333333333333, "endtime": 0.75},
    {"route_id": "w3", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.28125, "endtime": 0.6979166666666666},
    {"route_id": "w4", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.25, "endtime": 0.6666666666666666},
    {"route_id": "w5", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.2916666666666667, "endtime": 0.7083333333333334},
    {"route_id": "w6", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.22916666666666666, "endtime": 0.6458333333333334},
    {"route_id": "w7", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.3020833333333333, "endtime": 0.71875},

    {"route_id": "r1", "distance_total": 285.0, "distance_toll": 259.0, "starttime": 0.75, "endtime": 0.9375},
    {"route_id": "r2", "distance_total": 250.0, "distance_toll": 220.0, "starttime": 0.6875, "endtime": 0.90625},
    {"route_id": "r3", "distance_total": 235.0, "distance_toll": 219.0, "starttime": 0.7395833333333334, "endtime": 0.8958333333333334},

    {"route_id": "h3", "distance_total": 180.0, "distance_toll": 160.0, "starttime": 0.78125, "endtime": 0.9479166666666666},
    {"route_id": "h4", "distance_total": 180.0, "distance_toll": 160.0, "starttime": 0.7708333333333334, "endtime": 0.9375},

    {"route_id": "k1", "distance_total": 275.0, "distance_toll": 235.0, "starttime": 0.6875, "endtime": 0.9375},
]
routes_df = pd.DataFrame(routes_data)

# Initialisierung des Modells

In [ ]:
model = Model("Stufe3_Diesel-Elektro_Optimierung")

# 1. Indexmengen

Tour

In [ ]:
ROUTEN = routes_df.index.tolist()   # Menge aller Touren

In [ ]:
FUHRPARK = range(1, len(ROUTEN)+1)  # Alle geleasten LKWs im Fuhrpark

In [ ]:
FTypen = ["Diesel", "E400", "E600"] # Verfügbare LKW-Typen

In [ ]:
ZEIT = range(0, 96)  # Zeitschritte in 15-Minuten-Intervallen

In [ ]:
STypen = ["Alpitronic-50", "Alpitronic-200", "Alpitronic-400"]

In [ ]:
LADEPUNKT = [1, 2]

In [ ]:
SÄULENBESTAND = [1, 2, 3]

# 2. Parameter

LKW

In [ ]:
# Investitonskosten von LKW-Typ t
C_CAPEX_LKW = {"Diesel": 24000.0,
           "E400": 50000.0,
           "E600": 60000.0
}

# Operative Kosten von LKW-Typ t
C_OPEX_LKW = {"Diesel": 6000.0,
          "E400": 5000.0,
          "E600": 6000.0
}

# Engerieverbrauch der LKW-Typ t
VERBRAUCH = {"Diesel": 26.0/100,
             "E400": 105.0/100,
             "E600": 110.0/100
}

# Batteriekapazität
BATTERIE = {"Diesel": 0.0,
            "E400": 414.0,
            "E600": 621.0
            }

# Steuerkosten von Fahrzeugtyp t
C_STEUER = {"Diesel": 556.0,
            "E400": 0.0,
            "E600": 0.0
}

# Erlös THG von Fahrzeugtyp t
THG = {"Diesel": 0.0,
       "E400": 1000.0,
       "E600": 1000.0
}

# Power Limit, max. Ladeleistung von LKW-Typ t
POWER_LIMIT_LKW = {"Diesel": 0.0,
          "E400": 400.0,
          "E600": 400.0
}

Preise

In [ ]:
# Kraftstoffpreis
P_DIESEL = 1.60

# Stromkosten pro kwH
P_STROM = 0.25

Tour

In [ ]:
#Gesamte Fahrstrecke der Tour r
DISTANZ_GESAMT = {r: routes_df.loc[r, 'distance_total'] for r in ROUTEN}  #Wir schauen in jeder Spalte Gesamtdistanz in der Tabelle von den Routen und nehmen das für jede ROute raus

#Mautpflichtige STrecke der Tour r
DISTANZ_MAUT = {r: routes_df.loc[r, 'distance_toll'] for r in ROUTEN}  #siehe oben

# Mautgebühr
C_MAUT = 0.34

Zeit

In [ ]:
#Startzeitpunkt der Tour r
START = {r: routes_df.loc[r, 'starttime'] * 96 for r in ROUTEN}

#Endzeitpunkt der Tour r
END = {r: routes_df.loc[r, 'endtime'] * 96 for r in ROUTEN}

#Einsatztage pro Jahr
N_TAGE = 260

#Hilfsvariable ->läuft Route r zu welchen zeitpunkt z
from collections import defaultdict
ROUTE_AKTIV = defaultdict(int)
for r in ROUTEN:
    for z in ZEIT:
        ROUTE_AKTIV[r, z] = 1 if START[r] <= z < END[r] else 0    #Ende kommt raus, weil sonst haben wir anstatt 27 28 Slots (nahc ENde kommt ja kein Slot mehr) + verbruacht zu viel Energie

Ladeinfrastrukut

In [ ]:
# Jährliche Kosten Ladeinfrastruktur (Capex + Opex)
C_CAPEX_S = {
    "Alpitronic-50": 3000.0,
    "Alpitronic-200": 10000.0,
    "Alpitronic-400": 16000.0
}

C_OPEX_S = {
    "Alpitronic-50": 1000.0,
    "Alpitronic-200": 1500.0,
    "Alpitronic-400": 2000.0
}

# Power Limit, max. Ladeleistung vom Säulentyp S
POWER_LIMIT_S = {"Alpitronic-50": 50.0,
        "Alpitronic-200": 200.0,
        "Alpitronic-400": 400.0
}

C_GEBÜHR = 1000


#Anzahl Ladepunkte pro Säule
#LADEPUNKTE = {"Alpitronic-50": 2,
#      "Alpitronic-200": 2,
#      "Alpitronic-400": 2
#}

# 3. Entscheidungsvariablen

LKW

In [ ]:
# a[lkw, tour]: 1, wenn LKW f die Tour r fährt
lkw_fährt_r = {}
for f in FUHRPARK:
    for r in ROUTEN:
        lkw_fährt_r[f, r] = model.addVar(vtype="B", name=f"fahrt_{f}_{r}")

In [ ]:
# y[lkw, typ]: 1, wenn LKW f den Typ t hat
lkw_hat_typ = {}
for f in FUHRPARK:
    for t in FTypen:
        lkw_hat_typ[f, t] = model.addVar(vtype="B", name=f"kauf_{f}_{t}")

In [ ]:
# Bedeutung: LKW f vom Typ t fährt Route r (z[f,r,t] ersetzt das Produkt (a * y))
lkw_fährt_Route_mit_typ = {}
for f in FUHRPARK:
   for r in ROUTEN:
       for t in FTypen:
           lkw_fährt_Route_mit_typ[f, r, t] = model.addVar(vtype="B", name=f"z_{f}_{r}_{t}")


In [ ]:
# LKW f lädt  zum Zeitpunkt z an Säule s an Ladepunkt l
lkw_lädt = {}
for f in FUHRPARK:
    for z in ZEIT:
        for s in STypen:
            for l in LADEPUNKT:
              lkw_lädt[f, z, s, l] = model.addVar(vtype="B", name=f"load_{f}_{z}_{s}")

In [ ]:
# Batteriestand LKW f zum Zeitpunkt z
soc = {}
for f in FUHRPARK:
    for z in ZEIT:
        soc[f, z] = model.addVar(vtype="C", lb=0, name=f"soc_{f}_{z}")

In [ ]:
# n[s]: Anzahl der installierten Ladesäulen vom Typ s
anzahl_Säule = {}
for s in STypen:
    anzahl_Säule[s] = model.addVar(vtype="I", lb=0, name=f"n_{s}")

In [ ]:
# x[typ]: Anzahl der LKWs f pro Typ t
anzahl_lkw = {}
for t in FTypen:
    anzahl_lkw[t] = model.addVar(vtype="I", name=f"anzahl_{t}")

# 4. Zielfunktion

In [ ]:
# TEIL A: FIXKOSTEN (Was uns die LKWs pro Jahr kosten, egal wie viel sie fahren)
# Wir summieren für jeden Typ (Diesel, E400, E600):
# Grundgebühr Strom + (Anzahl LKWs) * (Leasing + Wartung + Steuer - THG-Bonus)
fix_kosten_lkw = C_GEBÜHR + quicksum(anzahl_lkw[t] * (C_CAPEX_LKW[t] + C_OPEX_LKW[t] + C_STEUER[t] - THG[t]) for t in FTypen)

# TEIL B: VARIABLE KOSTEN (Sprit/Strom & Maut insgesamt für alle Tage)
variable_kosten_total = quicksum(
    # Diesel Kosten
    lkw_fährt_Route_mit_typ[f, r, "Diesel"] * (DISTANZ_GESAMT[r] * VERBRAUCH["Diesel"] * P_DIESEL + DISTANZ_MAUT[r] * C_MAUT) +

    # E400 Kosten
    lkw_fährt_Route_mit_typ[f, r, "E400"] * DISTANZ_GESAMT[r] * VERBRAUCH["E400"] * P_STROM +

    # E600 Kosten
    lkw_fährt_Route_mit_typ[f, r, "E600"] * DISTANZ_GESAMT[r] * VERBRAUCH["E600"] * P_STROM
    for f in FUHRPARK for r in ROUTEN
)

# Kosten Ladeninfrastruktur
kosten_ladeinfrastruktur = quicksum( anzahl_Säule[s] * (C_CAPEX_S[s] + C_OPEX_S[s]) for s in STypen )

# GESAMTZIEL: Fixkosten + Variable Kosten (summiert über alle Tage) + Kosten Ladeinfrastruktur
model.setObjective(
    fix_kosten_lkw + variable_kosten_total * N_TAGE + kosten_ladeinfrastruktur,
    "minimize")

# 5. Nebenbedingungen

In [ ]:
# NB 1: LINEARE NEBENBEDINGUNGEN (damit gilt a[f,r] = 1  ⇔  Σt z[f,r,t] ≥ 1)
for f in FUHRPARK:
     for r in ROUTEN:
         for t in FTypen:
                # wenn lkw_fährt_Route_mit_typ =1, dann muss lkw_fährt_r 1        (wenn ===, dann gilt das auch gleichzeitig mit allen Typen)
            model.addCons(
                lkw_fährt_Route_mit_typ[f, r, t] <= lkw_fährt_r[f, r]
                )

                # wenn lkw_fährt_Route_mit_typ =1, dann muss lkw_hat_typ 1
            model.addCons(
                lkw_fährt_Route_mit_typ[f, r, t] <= lkw_hat_typ[f, t],
                          name=f"Link_zfdrt_y_{f}_{r}_{t}")

# Ensure a[f,r] is 1 if route r is driven by truck f
for f in FUHRPARK:
    for r in ROUTEN:
        model.addCons(
            lkw_fährt_r[f, r] <= quicksum(lkw_fährt_Route_mit_typ[f, r, t] for t in FTypen),
            name=f"Activate_a_{f}_{r}")


In [ ]:
# NB 2: Tourenabdeckung -> Jede Tour muss genau einmal gefahren werden
for r in ROUTEN:
    model.addCons(
        quicksum(lkw_fährt_r[f, r] for f in FUHRPARK) == 1
        )

In [ ]:
# NB 3: Typ-Eindeutigkeit -> Ein LKW f muss genau einem Typ entsprechen
for f in FUHRPARK:
    model.addCons(
        quicksum(lkw_hat_typ[f, t] for t in FTypen) <= 1          #wenn wir das nicht haben, kauft das Programm "Geister-LKWs" weil die nichts kosten weil wir wollen kosten minimieren (Unsere Kosten an den Typ gekoppelt, wenn LKW keinen Typ hat, hat er auch keine Kosten)
        )

In [ ]:
# NB 4: Kauf-Kopplung (Fahren nur bei Kauf) -> LKW (ein bisschen redundant, weil jeder LKW einem Typ zugewiesen wird, aber drinne lassen, damit Schlupflöcher weg)
for f in FUHRPARK:
        model.addCons(
            quicksum(lkw_fährt_r[f, r] for r in ROUTEN) <= len(ROUTEN) * quicksum(lkw_hat_typ[f, t] for t in FTypen)        # Wenn rechts d.h. y=0 ist dann hat LKW nicht Typ t, aber dann muss links a (=LkW fährt TOUR) auch 0 sein also LKW fährt nicht weil <=
            )

In [ ]:
# NB 5: Zeit-Check (Keine Tour-Überlappung pro LKW)
for f in FUHRPARK: #für jeden LKW im FUhrpark
    for r in ROUTEN:  #(+nächste Zeile:) schaue mir jedes Paar von den Routen an also r=Route1 und k = Route2
        for k in ROUTEN:
          # Dass jedes Paar nur einmal geprüft wird
          if r < k and START[k] < END[r] and START[r] < END[k]:       #Route k startet bevor Route r endet UND Route r startet bevor Route k endet
                    model.addCons(
                        lkw_fährt_r[f, r] + lkw_fährt_r[f, k] <= 1
                        )

In [ ]:
# NB 6: Flottenberechnung -> Anzahl LKWs pro Typ ist genauso groß wie die SUmme aus allen Fahrzeugen die einen Typ haben
for t in FTypen:
    model.addCons(
        anzahl_lkw[t] == quicksum(lkw_hat_typ[f, t] for f in FUHRPARK)
        )


In [ ]:
# NB 7: LKW nur laden, wenn er nicht auf Tour ist - Zeitliche Überlappung verhindern
for f in FUHRPARK:
    for z in ZEIT:
      model.addCons(quicksum(lkw_fährt_Route_mit_typ[f, r, t] * ROUTE_AKTIV[r, z] for r in ROUTEN for t in FTypen) + quicksum(lkw_lädt[f, z, s, l] for s in STypen for l in LADEPUNKT) <= 1)      # Der LKW kann entweder fahren oder laden

In [ ]:
# NB 8: Laden wird nur für die ELKW erlaubt, Diesel darf nicht laden
for f in FUHRPARK:
    for z in ZEIT:
        for s in STypen:
          for l in LADEPUNKT:
            model.addCons(lkw_lädt[f, z, s, l] <= lkw_hat_typ[f, "E400"] + lkw_hat_typ[f, "E600"])

In [ ]:
# NB 10: Reichweiten-Check & Ladepflicht
for f in FUHRPARK:
    for r in ROUTEN:
        # Startzeitpunkt der Route bestimmen (erster Zeitschritt mit U_rz == 1)
        start_schritte = [z for z in ZEIT if ROUTE_AKTIV[r, z] > 0.5]
        if start_schritte:

          z_start = min(start_schritte)

          for t in FTypen:
            if t != "Diesel":
                # Benötigte Energie für die gesamte Tour r
                bedarf = DISTANZ_GESAMT[r] * VERBRAUCH[t]

                # Wenn LKW f vom Typ t ist und Tour r fährt (z_frt == 1),
                # dann MUSS der SOC am Startpunkt >= Bedarf sein.
                # Wir nutzen ein "Big M" (hier 1000), um die Bedingung zu aktivieren:
                model.addCons(
                    soc[f, z_start] >= bedarf - (1 - lkw_fährt_Route_mit_typ[f, r, t]) * 1000,
                    name=f"Check_Reichweite_LKW{f}_Tour{r}"
                )

# NB 12: Erzwungenes Laden (Zusatz-Logik)
# Damit der LKW nicht "vergisst" zu laden, wenn er Zeit hat:
for f in FUHRPARK:
    for z in ZEIT:
        # Wenn der LKW ein E-LKW ist, darf er nie unter 10% (Sicherheitsreserve)
        model.addCons(
            soc[f, z] >= 0.1 * quicksum(lkw_hat_typ[f, t] * BATTERIE[t] for t in ["E400","E600"]),
            name=f"Sicherheitsreserve_f{f}_z{z}"
        )

In [ ]:
# NB 9: Energiefluss (SOC) für einen repräsentativen Standardtag (96 Schritte)
# Annahme: PL_LKW ist ein Dictionary mit der max. Ladeleistung der LKW-Modelle
# z.B. PL_LKW = {"eActros400": 160, "eActros600": 400, "Diesel": 0}

for f in FUHRPARK:
    # NB: Der SOC am Anfang (Schritt 0) muss exakt dem SOC am Ende (Schritt 95) entsprechen -> Das ist deine Bedingung: "Tag 2 Zeit 0 == Tag 1 Zeit 95"
    model.addCons(
        soc[f, 0] == soc[f, 95],
        name=f"Zyklus_SOC_LKW_{f}")

    for z in ZEIT:
        # Kapazitätsgrenze: SOC darf nie über die Batteriekapazität steigen
        model.addCons(soc[f, z] <= quicksum(lkw_hat_typ[f, t] * BATTERIE[t] for t in FTypen))

        if z > 0:
            # 1. VERBRAUCH (Entnahme)
            entnahme = quicksum(
                lkw_fährt_Route_mit_typ[f, r, t] * ((DISTANZ_GESAMT[r] * VERBRAUCH[t]) /
                max(1, sum(ROUTE_AKTIV[r, zz] for zz in ZEIT))) * ROUTE_AKTIV[r, z]
                for r in ROUTEN for t in FTypen if t != "Diesel"
            )

            # 2. LADUNG (Zufuhr) mit Leistungsvergleich (Min-Prinzip)
            # Wir berechnen die mögliche Zufuhr basierend auf der Säulenleistung
            # und stellen dann sicher, dass der LKW nicht mehr aufnimmt als sein eigenes Limit
            zufuhr = quicksum(lkw_lädt[f, z, s, l] * (POWER_LIMIT_S[s] / 4)
                              for s in STypen for l in LADEPUNKT)

            # Zusätzliche NB: Die Zufuhr darf das Limit des LKWs nicht überschreiten
            model.addCons(
                zufuhr <= quicksum(lkw_hat_typ[f, t] * (POWER_LIMIT_LKW.get(t, 0) / 4) for t in FTypen),
                name=f"Ladeleistung_Limit_f{f}_z{z}"
            )

            # 3. ENERGIEBILANZ
            # Der SOC im aktuellen Schritt ist der alte Stand minus Verbrauch plus Ladung
            model.addCons(
                soc[f, z] == soc[f, z-1] - entnahme + zufuhr,
                name=f"SOC_Fluss_f{f}_z{z}")

In [ ]:
# 9 Ladepunkte-Limit: Anzahl der geladenen LKWs darf Anzahl Ladepunkte nicht überschreiten
for z in ZEIT:
  for s in STypen:
        model.addCons(quicksum(lkw_lädt[f, z, s, l] for f in FUHRPARK) <= quicksum(anzahl_Säule[s] for l in LADEPUNKT))

In [ ]:
# NB. Max. dürfen drei Ladesäulen angeschafft werden
model.addCons(
    quicksum(anzahl_Säule[s] for s in STypen) <= 3
    )

c27432

In [ ]:
# NB: Erzwungene volle Auslastung zu jedem Zeitpunkt z (es hängt immer ein LKW an der Ladesäule)
#for z in ZEIT:
 #   for s in STypen:
  #      # Die Summe der ladenden LKWs darf höchstens der Anzahl der Ladepunkte entsprechen
   #     model.addCons(
    #        quicksum(lkw_lädt[f, z, s] for f in FUHRPARK) <= quicksum(anzahl_Säule[s] for l in LADEPUNKT),
     #       name=f"Full_Utilization_z{z}_s{s}"
      #  )

In [ ]:
# Ein LKW kann zur Zeit z maximal an EINER Säule laden
for f in FUHRPARK:
    for z in ZEIT:
      for l in LADEPUNKT:
        model.addCons(quicksum(lkw_lädt[f, z, s, l] for s in STypen) <= 1)

Modell optimieren

In [ ]:
model.setRealParam("limits/time", 300) # Gib ihm 5 Minuten Zeit
model.optimize()
print(f"Solver Status: {model.getStatus()}")

Solver Status: timelimit


Ausgabe

In [ ]:
if model.getNSols() > 0:
    print("\n" + "="*50)
    print(f"ERGEBNIS: {model.getObjVal():,.2f} € Gesamtkosten/Jahr")
    print("="*50)

    # 1. ANZAHL UND TYP DER LKWS
    print("\n--- FLOTTENZUSAMMENSETZUNG ---")
    for t in FTypen:
        anzahl = int(model.getVal(anzahl_lkw[t]))
        if anzahl > 0:
            # Wir nutzen die Parameter aus deinem Modell für die Anzeige
            print(f"➤ {t:8}: {anzahl} Fahrzeug(e)")

    # 2. ZUORDNUNG DER TOUREN
    print("\n--- EINSATZPLAN PRO LKW-SLOT ---")
    gefundene_lkws = 0
    for f in FUHRPARK:
        # Prüfen, welche Touren diesem Slot f zugeordnet sind
        tours_for_f = [routes_df.loc[r, 'route_id'] for r in ROUTEN if model.getVal(lkw_fährt_r[f, r]) > 0.5]

        if tours_for_f:
            gefundene_lkws += 1
            # Herausfinden, welcher Typ für diesen Slot gewählt wurde
            lkw_typ = "Unbekannt"
            for t in FTypen:
                if model.getVal(lkw_hat_typ[f, t]) > 0.5:
                    lkw_typ = t
                    break

            # Schöne Auflistung der Touren
            tour_liste = ", ".join(tours_for_f)
            print(f"[{lkw_typ:6}]: fährt Tour(en) -> {tour_liste}")

            # 2. BESCHAFFTE LADEINFRASTRUKTUR

    print("\n--- LADEINFRASTRUKTUR ---")
    infrastruktur_vorhanden = False
    for s in STypen:
        anzahl_s = int(round(model.getVal(anzahl_Säule[s])))
        if anzahl_s > 0:
            infrastruktur_vorhanden = True
            print(f"➤ {s:15}: {anzahl_s} Säule(n) beschafft")
    if not infrastruktur_vorhanden:
        print(" Keine Ladesäulen benötigt (reiner Diesel-Betrieb).")

    besetzte_slots = 0
    total_kapazitaet = 0

    for z in ZEIT:
        for s in STypen:
            anzahl_s = int(round(model.getVal(anzahl_Säule[s])))
            total_kapazitaet += sum(anzahl_s for l in LADEPUNKT) # Changed quicksum to sum
            for f in FUHRPARK:
                for l in LADEPUNKT: # Added loop for LADEPUNKT to correctly access lkw_lädt
                    if model.getVal(lkw_lädt[f, z, s, l]) > 0.5:
                        besetzte_slots += 1

    if 1 <= total_kapazitaet:
        auslastung = (besetzte_slots / total_kapazitaet) * 100
        print(f"Durchschnittliche Auslastung der Ladepunkte: {auslastung:.2f}%")

    print("\n" + "="*80)
    print(f"{'Uhrzeit':<8} | {'LKW':<8} | {'Typ':<8} | {'Aktivität':<20} | {'Batterie'}")
    print("-" * 80)

    for f in FUHRPARK:
        lkw_typ = next((t for t in FTypen if model.getVal(lkw_hat_typ[f, t]) > 0.5), None) # Changed y to lkw_hat_typ
        if not lkw_typ: continue

        last_status = None
        for z in ZEIT:
            uhrzeit = f"{int((z*15)//60):02d}:{int((z*15)%60):02d}"
            soc_val = model.getVal(soc[f, z])

            # Status-Check
            status = "Standby"
            for r in ROUTEN:
                if model.getVal(lkw_fährt_r[f, r]) > 0.5 and ROUTE_AKTIV[r, z] > 0.5: # Changed a to lkw_fährt_r
                    status = f"🚚 Tour {r}"
                    break
            if status == "Standby":
                for s in STypen:
                    for l in LADEPUNKT: # Added loop for LADEPUNKT to correctly access lkw_lädt
                        if model.getVal(lkw_lädt[f, z, s, l]) > 0.5:
                            status = f"⚡ Laden ({s})"
                            break
                    if status != "Standby": # Break inner loop if charging found
                        break

            # Nur drucken bei Änderung
            if status != last_status:
                b_anzeige = f"{soc_val:6.1f} kWh" if lkw_typ != "Diesel" else "---"
                print(f"{uhrzeit:<8} | LKW {f:02d} | {lkw_typ:<8} | {status:<20} | {b_anzeige}")
                last_status = status
else:
    print("Keine Lösung gefunden.")


ERGEBNIS: 19,927,519.60 € Gesamtkosten/Jahr

--- FLOTTENZUSAMMENSETZUNG ---
➤ Diesel  : 13 Fahrzeug(e)
➤ E600    : 1 Fahrzeug(e)

--- EINSATZPLAN PRO LKW-SLOT ---
[Diesel]: fährt Tour(en) -> t-4, r3
[Diesel]: fährt Tour(en) -> t-5, r1
[Diesel]: fährt Tour(en) -> s-1, h4
[Diesel]: fährt Tour(en) -> s-3
[Diesel]: fährt Tour(en) -> w6
[Diesel]: fährt Tour(en) -> w5
[Diesel]: fährt Tour(en) -> w7
[Diesel]: fährt Tour(en) -> w4
[Diesel]: fährt Tour(en) -> w2
[Diesel]: fährt Tour(en) -> s-2
[Diesel]: fährt Tour(en) -> w1, h3
[Diesel]: fährt Tour(en) -> s-4, k1
[E600  ]: fährt Tour(en) -> t-6, r2
[Diesel]: fährt Tour(en) -> w3

--- LADEINFRASTRUKTUR ---
 Keine Ladesäulen benötigt (reiner Diesel-Betrieb).

Uhrzeit  | LKW      | Typ      | Aktivität            | Batterie
--------------------------------------------------------------------------------
00:00    | LKW 01 | Diesel   | Standby              | ---
06:45    | LKW 01 | Diesel   | 🚚 Tour 0             | ---
17:15    | LKW 01 | Diesel   